In [1]:
import numpy as np

data = np.genfromtxt('data/ex1/ex1data2.txt', delimiter=',')
input_X = data[:, 0:2].astype(np.float32)
input_y = data[:, 2:3].astype(np.float32)

data[:10]

array([[  2.10400000e+03,   3.00000000e+00,   3.99900000e+05],
       [  1.60000000e+03,   3.00000000e+00,   3.29900000e+05],
       [  2.40000000e+03,   3.00000000e+00,   3.69000000e+05],
       [  1.41600000e+03,   2.00000000e+00,   2.32000000e+05],
       [  3.00000000e+03,   4.00000000e+00,   5.39900000e+05],
       [  1.98500000e+03,   4.00000000e+00,   2.99900000e+05],
       [  1.53400000e+03,   3.00000000e+00,   3.14900000e+05],
       [  1.42700000e+03,   3.00000000e+00,   1.98999000e+05],
       [  1.38000000e+03,   3.00000000e+00,   2.12000000e+05],
       [  1.49400000e+03,   3.00000000e+00,   2.42500000e+05]])

In [2]:
import tensorflow as tf

train_X = tf.placeholder(dtype=input_X.dtype, shape=input_X.shape, name='train_X')
X = tf.placeholder(tf.float32, shape=[None, 2], name='X')
y = tf.placeholder(tf.float32, shape=[None, 1], name='y')

with tf.name_scope('normalize_features'):
    mean, var = tf.nn.moments(train_X, axes=[0])
    mu = tf.Variable(mean, trainable=False, collections=[], name='mu')
    sigma = tf.Variable(tf.sqrt(var), trainable=False, collections=[], name='sigma')
    norm_X = (X - mu) / sigma

In [3]:
print(train_X.shape, mean.shape, var.shape)

(47, 2) (2,) (2,)


In [4]:
weights = tf.Variable(tf.zeros([2, 1]), dtype=tf.float32, name='weights')
bias = tf.Variable(tf.zeros([1]), dtype=tf.float32, name='bias')

with tf.name_scope('hypothesis'):
    h = tf.matmul(norm_X, weights) + bias

with tf.name_scope('cost'):
    square_delta = tf.square(h - y)
    loss = tf.reduce_mean(square_delta) / 2

In [5]:
alpha = 0.1
optimizer = tf.train.GradientDescentOptimizer(alpha)
train = optimizer.minimize(loss)

In [6]:
sess = tf.Session()

sess.run([mu.initializer, sigma.initializer], feed_dict={train_X: input_X})
init = tf.global_variables_initializer()
sess.run(init)

In [7]:
import os
from datetime import datetime

def subdir(base='logs'):
    return os.path.join(base, datetime.now().strftime('%Y%m%d-%H%M%S'))

tf.summary.scalar('loss', loss)
merged = tf.summary.merge_all()

logdir = 'logs/ex1_multi'
writer = tf.summary.FileWriter(subdir(logdir), sess.graph)

In [8]:
feed_dict = {X: input_X, y: input_y}
max_iterations = 400
for i in range(max_iterations):
    if i % 10 == 0:
        summary, _ = sess.run([merged, loss], feed_dict=feed_dict)
        writer.add_summary(summary, i)
    sess.run(train, feed_dict=feed_dict)

In [9]:
def predict(x):
    if not hasattr(x, 'reshape'):
        x = np.array(x)
    x = x.reshape(-1, 2)
    return sess.run(h, feed_dict={X: x})

price = predict([1650, 3])
print('Predicted price of a 1650 sq-ft, 3 br house (using gradient descent):',
      '\n $%f\n' % price)

Predicted price of a 1650 sq-ft, 3 br house (using gradient descent): 
 $293081.343750



In [10]:
writer.close()